In [ ]:
# 9.9.21
# First let's plot the ecoblock graph

In [1]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot
import csv  # jaimie do you need this?

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [2]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
#modelpath = filepath + "ecoblock_impedexcel_trueZ2.xls"
modelpath = filepath + "OaklandJ_impedexcel_short.xlsx"

# load data is not actually used in assessment, but the feeder object builds variables to hold load info
loadfolder = filepath
loadpath = filepath + 'ecoblock_NL_PVpen100_sigbuilder.csv'
headerpath = filepath + 'ecoblock_time_header.csv'
load_data = loadpath

#file_name = string specifying name of dot file created when make_graph() is called
file_name = 'oaklandJ'
#file_name = 'ecoblock'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_nSOURCE' # formatted as 'bus_XXX'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2022-01-02 19:03:30.163408


In [3]:
# [ESSENTIAL] create feeder object and plot feeder with make_graph func

fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

#print('depths=',depths) # check this is populated, lists how far each node is from substation
print('depths length=',len(depths))

# print list of first 10 buses in network
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
graph = fin_feeder.network
print('node index map len=',len(node_index_map))

Finished initializing feeder
depths length= 239
bus_n100
bus_n101
bus_n102
bus_n103
bus_n104
bus_n105
bus_n106
bus_n107
bus_n108
bus_n109
node index map len= 239


In [ ]:
# mark actuators on ecoblock graph
lst_act_locs=['bus_4','bus_12','bus_3079','bus_3089','bus_3011','bus_3015','bus_3025','bus_3029','bus_3033','bus_3037','bus_3038','bus_3044','bus_3049','bus_3056','bus_3058','bus_3057','bus_3059','bus_3063','bus_3062','bus_3001','bus_3008']
#[4 3079 3089 3011 3015 3025 3029 3033 3037 3038 3044 3049 3056 3058 3057 3059 3063 3062 3001 3008];
vis.markActuatorConfig(lst_act_locs, fin_feeder, file_name)

In [ ]:
# gather impedance info Xii for each actuator
# read in sigbuilder csv. For each col (phase-act) up to half of all cols, remove take substring before / delimiter, append bus_, then add to cby_load

opened_csv_file = open('feeder_impedance_models/OaklandJ_sigbuilder.csv', 'r')
reader = csv.reader(opened_csv_file)

loads_table = []
for row in reader:
        loads_table.append(row)
#print(loads_table[0]) # header
lst_act_locs=[]
for phaseAct_name in loads_table[0]:
    str1 = phaseAct_name.split("/",1)[0] # LD_nXXX
    str2=str1.replace('LD_', 'bus_') # bus_nXXX
    lst_act_locs.append(str2) 
lst_act_locs.pop(0) # remove 'Time' from list
print(lst_act_locs[0:5])

# compute impedances to substation
x_acts=[] # separate  r and x so easier to create complex number in matlab
r_acts=[]
for cby_load in lst_act_locs:
    Zii=imp.get_total_impedance_from_substation(fin_feeder, cby_load, depths) # returns dict of 3x3 numpy array for Z_ii
    print('zii=',Zii[0,0])
    x_acts.append(np.imag(Zii[0,0]))
    r_acts.append(np.real(Zii[0,0]))
    #print('zii of ',cby_load,' = ',Zii)
print(x_acts[0:5])

# now write lst_act_locs to first row of csv, and Zii to second row of csv
with open('oaklandJ_zToSub.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(loads_table[0]) # same header as sigbuilder
    writer.writerow(r_acts)
    writer.writerow(x_acts)

In [ ]:
# set config using lst_act_locs, and call Gdisc kgain design func. The func does:
# for 3ph system there are r phase-actuators, for example node6_phA; thus there are r nonzero F elements
# for each of nonzero element of F, det Gdisc conditions; conditions give an upper bound on that F ele search space 
# compute vector of F_ub, dimension rx1
# search space of [0 F_ub] in r-space and det evals of closed-loop system
# Select combo with smallest spectral radius, and return kgains of dim rx1


In [ ]:
# print kgains vector to csv, and in ephasorsim we read in csv to assign to Kp_vmag, Ki_vmag, etc.
# kgains has dimension rx1, where is number of phase-actuators)

In [ ]:
# ------------- do a basic eval_config ----------------------------
parmObj=hm.configParms()
# parmObj.set_ctrlTypes(['VVC', 'VVC', 'VWC', 'VWC', 'VWC', 'VVC', 'VVC', 'VVC'])
# parmObj.set_version(2) # 1 for PBC

parmObj.set_ctrlTypes(['PBC', 'PBC'])
parmObj.set_version(1) # 1 for PBC

#all_act_locs = ['bus_152', 'bus_21', 'bus_58', 'bus_70', 'bus_4', 'bus_82', 'bus_109', 'bus_47']
all_act_locs = ['bus_n103','bus_n156']
perf_nodes=['bus_n103','bus_n156']
# print(len(all_act_locs))
# print(len(foo))
feas, maxError,numfeas,bestF,indicMat=hm.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [4]:
# ---------- Create cby_cand -------------
# it's the subset of OaklandJ nodes where can place actuator, i.e. MV edge nodes or one-up-from LV edge nodes
mygraph=fin_feeder.network
LVlist=[] # all nodes on low voltage side
for node_raw in mygraph.nodes(): # all nodes
    #print('noderaw=',node_raw)
    if 'SOURCE' in node_raw or 'n239' in node_raw:
        print('found source in ',node_raw)
    else:
        nodekey=int(node_raw.split("_n",2)[1]) # get latter of split, and convert string to numeral
        #print('nodekey=',nodekey)
        if nodekey<=235 and nodekey>=161:
            node_raw2='bus_n'+str(nodekey)
            LVlist.append(node_raw2)
print('num LV nodes=',len(LVlist))

leafNodes=[x for x in mygraph.nodes() if mygraph.out_degree(x)==0 and mygraph.in_degree(x)==1]
print('num leaf=',len(leafNodes))

cby_cand=[]
for bus in leafNodes:
    if 'n239' in bus:
        print('skipping n239, attached to root node')
    elif bus in LVlist:
        lst=list(mygraph.predecessors(bus)) # convert iterator to list, then index out of it
        cby_cand.append(lst[0]) # add one node up, which is on MV side
    else:
         cby_cand.append(bus) # add the MV node
            
print('num cand cby=',len(cby_cand))
#vis.markActuatorConfig(cby_cand, fin_feeder, file_name)

found source in  bus_n239
found source in  bus_nSOURCE
num LV nodes= 75
num leaf= 89
skipping n239, attached to root node
num cand cby= 88


In [ ]:
# Seed=3
# ------ run place_max_colocated_acts, aka auto_overloadCPP process -------------
t = time.time()
seed=3 # choose
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config,parmObj = hm.place_max_coloc_acts(parmObj,seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, 
                                                 Sbase, load_data, headerpath, modelpath, cby_cand)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# Seed=4
# ------ run place_max_colocated_acts, aka auto_overloadCPP process -------------
t = time.time()
seed=4 # choose
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config2,parmObj2 = hm.place_max_coloc_acts(parmObj,seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, 
                                                 Sbase, load_data, headerpath, modelpath, cby_cand)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# Seed=5
# ------ run place_max_colocated_acts, aka auto_overloadCPP process -------------
t = time.time()
seed=5 # choose
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config3,parmObj3 = hm.place_max_coloc_acts(parmObj,seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, 
                                                 Sbase, load_data, headerpath, modelpath, cby_cand)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# Seed=6
# ------ run place_max_colocated_acts, aka auto_overloadCPP process -------------
t = time.time()
seed=6 # choose
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config4,parmObj4 = hm.place_max_coloc_acts(parmObj,seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, 
                                                 Sbase, load_data, headerpath, modelpath, cby_cand)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# Seed=7
# ------ run place_max_colocated_acts, aka auto_overloadCPP process -------------
t = time.time()
seed=7 # choose
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config5,parmObj5 = hm.place_max_coloc_acts(parmObj,seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, 
                                                 Sbase, load_data, headerpath, modelpath, cby_cand)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# save max_act_config into csv to be read into matlab
with open('max_act_config_lst.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(max_act_config) # same header as sigbuilder
    writer.writerow(max_act_config2) # same header as sigbuilder
    writer.writerow(max_act_config3) # same header as sigbuilder
    writer.writerow(max_act_config4) # same header as sigbuilder
    writer.writerow(max_act_config5) # same header as sigbuilder

In [ ]:
# arb chosen config
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
parmObj.set_ctrlTypes(['PBC', 'PBC', 'PBC', 'PBC'])

all_act_locs = ['bus_n42','bus_n19','bus_n31','bus_n83']
perf_nodes= ['bus_n42','bus_n19','bus_n31','bus_n83']
feas, maxError,numfeas,bestF,indicMat=hm.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# manually chosen config
parmObj=hm.configParms()
parmObj.set_version(1) # 1 for PBC
parmObj.set_ctrlTypes(['PBC', 'PBC', 'PBC', 'PBC'])

all_act_locs = ['bus_n117','bus_n148','bus_n89','bus_n38']
perf_nodes= ['bus_n117','bus_n148','bus_n89','bus_n38']
feas, maxError,numfeas,bestF,indicMat=hm.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# # save configs+seed into .pkl
# # # Save RHP variable 'lst_feas_configs' to .pkl file, so dont have to run again in future
# import pickle
# filename='max_act_config_seed3.pkl'
# with open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(max_act_config, f)

In [13]:
# load the data 'lst_feas_configs' from .pkl file
# import pickle
# filename='nbhs_RHP_lstfeasconfigs.pkl'
# with open(filename,'rb') as f:  # Python 3: open(..., 'rb')
#     lst_feas_configs = pickle.load(f)
     
    
import os
import pickle
scores = {} # scores is an empty dict already
filename='max_act_config_seed3.pkl'
print(os.path.getsize(filename))
if os.path.getsize(filename) > 0:      
    with open(filename, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        max_act_config = unpickler.load()
        
print(len(max_act_config)) # just to see if it loaded

78
5


In [14]:
# -------- mark the max act configs for each seed
vis.markTwoActatorConfig(cby_cand,max_act_config,fin_feeder,file_name+'seed3')